In [4]:
from datetime import datetime, date
import calendar
import json 

def generate_monthly_bill(item_list: list, target_month: str) -> dict:
    year, month = map(int, target_month.split('-'))
    last_day = calendar.monthrange(year, month)[1]
    month_start = date(year, month, 1)
    month_end = date(year, month, last_day)
    
    groups = {}
    
    for item in item_list:
        start_date = datetime.strptime(item['start_date'], '%Y-%m-%d').date()
        stop_date = datetime.strptime(item['stop_date'], '%Y-%m-%d').date()
        
        # Calculate overlap with target month
        overlap_start = max(start_date, month_start)
        overlap_end = min(stop_date, month_end)
        if overlap_start > overlap_end:
            continue
        
        # Calculate active days
        active_days = (overlap_end - overlap_start).days + 1
        days_in_month = (month_end - month_start).days + 1
        
        # Convert qty and rate to floats
        qty = float(item['qty']) if isinstance(item['qty'], str) else float(item.get('qty', 0))
        rate = float(item['rate']) if isinstance(item['rate'], str) else float(item.get('rate', 0))
        
        # Calculate prorated qty
        prorated_qty = qty * active_days / days_in_month
        
        item_code = item['item_code']
        group_key = (item_code, rate, overlap_start, overlap_end)
        
        # Update the group
        if group_key not in groups:
            groups[group_key] = {
                'prorated_qty_sum': 0.0,
                'item_code': item_code,
                'rate': rate,
                'overlap_start': overlap_start,
                'overlap_end': overlap_end
            }
        groups[group_key]['prorated_qty_sum'] += prorated_qty
    
    line_items = []
    total_revenue = 0.0
    
    for key in groups:
        group = groups[key]
        prorated_qty_sum = group['prorated_qty_sum']
        qty = int(prorated_qty_sum) if prorated_qty_sum.is_integer() else round(prorated_qty_sum, 2)
        
        line_item = {
            "item_code": group['item_code'],
            "rate": group['rate'],
            "qty": qty,
            "amount": round(prorated_qty_sum * group['rate'], 2),
            "billing_period": f"{group['overlap_start'].strftime('%Y-%m-%d')} to {group['overlap_end'].strftime('%Y-%m-%d')}"
        }
        line_items.append(line_item)
        total_revenue += line_item['amount']
    
    return {
        "line_items": line_items,
        "total_revenue": round(total_revenue, 2)
    }

#item_list
item_list = [
    {
        "idx": 1,
        "item_code": "Executive Desk (4*2)",
        "sales_description": "Dedicated Executive Desk",
        "qty": 10,
        "rate": "1000",
        "amount": "10000",
        "start_date": "2023-11-01",
        "stop_date": "2024-10-17",
    },
    {
        "idx": 2,
        "item_code": "Executive Desk (4*2)",
        "qty": "10",
        "rate": "1080",
        "amount": "10800",
        "start_date": "2024-10-18",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 3,
        "item_code": "Executive Desk (4*2)",
        "qty": 15,
        "rate": "1080",
        "amount": "16200",
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 4,
        "item_code": "Executive Desk (4*2)",
        "qty": 5,
        "rate": "1000",
        "amount": "5000",
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 5,
        "item_code": "Manager Cabin",
        "qty": 5,
        "rate": 5000,
        "amount": 25000,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 6,
        "item_code": "Manager Cabin",
        "qty": 7,
        "rate": "5000",
        "amount": 35000,
        "start_date": "2024-12-15",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 7,
        "item_code": "Manager Cabin",
        "qty": 10,
        "rate": 4600,
        "amount": 46000,
        "start_date": "2023-11-01",
        "stop_date": "2024-10-17",
    },
    {
        "idx": 8,
        "item_code": "Parking (2S)",
        "qty": 10,
        "rate": 1000,
        "amount": 10000,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 9,
        "item_code": "Parking (2S)",
        "qty": 10,
        "rate": 0,
        "amount": 0,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 10,
        "item_code": "Executive Desk (4*2)",
        "qty": "8",
        "rate": "1100",
        "amount": "8800",
        "start_date": "2024-11-15",
        "stop_date": "2025-01-31",
    },
    {
        "idx": 11,
        "item_code": "Manager Cabin",
        "qty": "3",
        "rate": "5200",
        "amount": "15600",
        "start_date": "2024-10-10",
        "stop_date": "2024-11-10",
    },
    {
        "idx": 12,
        "item_code": "Conference Table",
        "qty": 1,
        "rate": "20000",
        "amount": "20000",
        "start_date": "2024-11-05",
        "stop_date": "2024-11-20",
    },
    {
        "idx": 13,
        "item_code": "Parking (2S)",
        "qty": 5,
        "rate": "1000",
        "amount": "5000",
        "start_date": "2024-11-15",
        "stop_date": "2025-02-28",
    },
    {
        "idx": 14,
        "item_code": "Reception Desk",
        "qty": 2,
        "rate": "7000",
        "amount": "14000",
        "start_date": "2024-11-01",
        "stop_date": "2025-03-31",
    },
    {
        "idx": 15,
        "item_code": "Reception Desk",
        "qty": 1,
        "rate": "7000",
        "amount": "7000",
        "start_date": "2024-11-10",
        "stop_date": "2024-11-25",
    },
    {
        "idx": 16,
        "item_code": "Breakout Area",
        "qty": 3,
        "rate": "3000",
        "amount": "9000",
        "start_date": "2024-01-01",
        "stop_date": "2024-01-31",
    }
]

# print the bill for November 2024
result = generate_monthly_bill(item_list, "2024-11")
print(json.dumps(result, indent=4))

{
    "line_items": [
        {
            "item_code": "Executive Desk (4*2)",
            "rate": 1080.0,
            "qty": 25,
            "amount": 27000.0,
            "billing_period": "2024-11-01 to 2024-11-30"
        },
        {
            "item_code": "Executive Desk (4*2)",
            "rate": 1000.0,
            "qty": 5,
            "amount": 5000.0,
            "billing_period": "2024-11-01 to 2024-11-30"
        },
        {
            "item_code": "Manager Cabin",
            "rate": 5000.0,
            "qty": 5,
            "amount": 25000.0,
            "billing_period": "2024-11-01 to 2024-11-30"
        },
        {
            "item_code": "Parking (2S)",
            "rate": 1000.0,
            "qty": 10,
            "amount": 10000.0,
            "billing_period": "2024-11-01 to 2024-11-30"
        },
        {
            "item_code": "Parking (2S)",
            "rate": 0.0,
            "qty": 10,
            "amount": 0.0,
            "billing_period": "202